In [1]:
import os

os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
os.environ["TORCH_USE_CUDA_DSA"] = "1"

In [2]:
%%time
import lib.utils.utils as utils
from run import get_images
#import lib.config as config
from lib.model.base_model import BaseModel
#import lib.roi as roi
#import lib.split_pages as spages
from lib.utils.promptLoader import PromptLoader

CPU times: user 606 ms, sys: 401 ms, total: 1.01 s
Wall time: 1min 4s


In [3]:
image_path = "images/cropped/The_Lightfoot_Herbarium_05_cropped_2.jpg"
image_dir = "images/cropped/"
default_yaml = "./prompts/default_v2_fromtext.yaml"
conversation = None

In [4]:
print(image_path)

images/cropped/The_Lightfoot_Herbarium_05_cropped_2.jpg


In [5]:
images = sorted(utils.load_images(image_dir))

In [6]:
model = BaseModel("qwen_model", prompt=default_yaml, conversation=conversation, max_new_tokens = 130000, batch_size=1, temperature=1, save_path="./outputs/lightfootcat/")

/mnt/shared/scratch/ikarunak/apps/conda/envs/lightfootcat/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
`Qwen2VLRotaryEmbedding` can now be fully parameterized by passing the model config through the `config` argument. All other arguments will be removed in v4.46
Loading checkpoint shards: 100%|██████████| 5/5 [08:21<00:00, 100.26s/it]


In [7]:
#outputs = model(images, save=False, debug=False, save_file_name="two_images")

In [6]:
from lib.utils.text_utils import convertToTextBlocks
#extracted_text = model.extract_text(images)
with open("./temp.txt", "r") as f_:
    extracted_text = f_.read()

text_blocks = convertToTextBlocks(extracted_text)

In [13]:
#text_blocks["Dicotyledones"]

In [15]:
large_blocks = []
for key in text_blocks.keys():
    print(key)
    for item in text_blocks[key]:
        if len(item) > 12000:
            large_blocks.append(item)
            print("="*10)
            print(item)
            print("="*10)

Dicotyledones
COMPOSITAE

Anaphalis margaritacea (L.) Benth. 2 folders.
Folder 1. Gnaphalium margaritaceum [G]. i. "Gnaph: armeria s:d to

be British. Bulls: [Bulstrode Park]" [JL].
Folder 2. Gnaphalium margaritaceum [TA].

Antennaria dioica (L.) Gaertn. 3 folders.
Folder 1. Gnaphalium dioicum [TA]. i. "Newfoundland [A. neodioica Greene] Mr. Banks" [JL]. ii. "Gnaphalium dioicum on Stratton Heath ab' 7 miles from Norwich. Mr. John Pitchford" [B]. Folder 2. Gnaphalium dioicum [G]. i. "Gnaph: dioic: mas & foem. one Specimen was 15 Inches high. a var: only" [JL]. Folder 3. Gnaphalium dioicum [TA].

Antennaria neodioica Greene [See under A. dioica]

Anthemis arvensis L. 1 folder. Anthemis arvensis [TA].

Anthemis cotula L. 1 folder. Anthemis cotula [TA]. i. "Anthemis cotula" [JL]. ii. "This I take to be the Anthemis arvensis by the Road Side going to Maidenhead" [JL].

Arnosera minima (L.) Schweigg. & Koerte 1 folder. Hyoseris minima [TA], Flora Scotica ed. 1, 442 (1777). i. "Hyoseris minim

In [50]:
import lib.utils.text_utils as tu
import re
def split_text_by_family(text, max_tokens=12000):
    
    # Define a regex to find family names (all caps followed by  a newline)
    family_regex = re.compile(r'([A-Z ]+\n\n|.+?[aA][cC][eE][aA][eE]\n\n)')
    
    # Split text by family names, keeping the delimiter
    parts = re.split(family_regex, text)
    parts = list(filter(None, parts))
    parts = list(zip(parts[::2], parts[1::2]))
    print(parts)
    print(len(parts))
    return parts
a = split_text_by_family(large_blocks[0])

[('COMPOSITAE\n\n', 'Anaphalis margaritacea (L.) Benth. 2 folders.\nFolder 1. Gnaphalium margaritaceum [G]. i. "Gnaph: armeria s:d to\n\nbe British. Bulls: [Bulstrode Park]" [JL].\nFolder 2. Gnaphalium margaritaceum [TA].\n\nAntennaria dioica (L.) Gaertn. 3 folders.\nFolder 1. Gnaphalium dioicum [TA]. i. "Newfoundland [A. neodioica Greene] Mr. Banks" [JL]. ii. "Gnaphalium dioicum on Stratton Heath ab\' 7 miles from Norwich. Mr. John Pitchford" [B]. Folder 2. Gnaphalium dioicum [G]. i. "Gnaph: dioic: mas & foem. one Specimen was 15 Inches high. a var: only" [JL]. Folder 3. Gnaphalium dioicum [TA].\n\nAntennaria neodioica Greene [See under A. dioica]\n\nAnthemis arvensis L. 1 folder. Anthemis arvensis [TA].\n\nAnthemis cotula L. 1 folder. Anthemis cotula [TA]. i. "Anthemis cotula" [JL]. ii. "This I take to be the Anthemis arvensis by the Road Side going to Maidenhead" [JL].\n\nArnosera minima (L.) Schweigg. & Koerte 1 folder. Hyoseris minima [TA], Flora Scotica ed. 1, 442 (1777). i. "Hyo

In [51]:
a

[('COMPOSITAE\n\n',
  'Anaphalis margaritacea (L.) Benth. 2 folders.\nFolder 1. Gnaphalium margaritaceum [G]. i. "Gnaph: armeria s:d to\n\nbe British. Bulls: [Bulstrode Park]" [JL].\nFolder 2. Gnaphalium margaritaceum [TA].\n\nAntennaria dioica (L.) Gaertn. 3 folders.\nFolder 1. Gnaphalium dioicum [TA]. i. "Newfoundland [A. neodioica Greene] Mr. Banks" [JL]. ii. "Gnaphalium dioicum on Stratton Heath ab\' 7 miles from Norwich. Mr. John Pitchford" [B]. Folder 2. Gnaphalium dioicum [G]. i. "Gnaph: dioic: mas & foem. one Specimen was 15 Inches high. a var: only" [JL]. Folder 3. Gnaphalium dioicum [TA].\n\nAntennaria neodioica Greene [See under A. dioica]\n\nAnthemis arvensis L. 1 folder. Anthemis arvensis [TA].\n\nAnthemis cotula L. 1 folder. Anthemis cotula [TA]. i. "Anthemis cotula" [JL]. ii. "This I take to be the Anthemis arvensis by the Road Side going to Maidenhead" [JL].\n\nArnosera minima (L.) Schweigg. & Koerte 1 folder. Hyoseris minima [TA], Flora Scotica ed. 1, 442 (1777). i. "H

In [52]:
family_name, data_ = a[0]

In [66]:
len(large_blocks[0])

15580

In [67]:
print(large_blocks[0])

COMPOSITAE

Anaphalis margaritacea (L.) Benth. 2 folders.
Folder 1. Gnaphalium margaritaceum [G]. i. "Gnaph: armeria s:d to

be British. Bulls: [Bulstrode Park]" [JL].
Folder 2. Gnaphalium margaritaceum [TA].

Antennaria dioica (L.) Gaertn. 3 folders.
Folder 1. Gnaphalium dioicum [TA]. i. "Newfoundland [A. neodioica Greene] Mr. Banks" [JL]. ii. "Gnaphalium dioicum on Stratton Heath ab' 7 miles from Norwich. Mr. John Pitchford" [B]. Folder 2. Gnaphalium dioicum [G]. i. "Gnaph: dioic: mas & foem. one Specimen was 15 Inches high. a var: only" [JL]. Folder 3. Gnaphalium dioicum [TA].

Antennaria neodioica Greene [See under A. dioica]

Anthemis arvensis L. 1 folder. Anthemis arvensis [TA].

Anthemis cotula L. 1 folder. Anthemis cotula [TA]. i. "Anthemis cotula" [JL]. ii. "This I take to be the Anthemis arvensis by the Road Side going to Maidenhead" [JL].

Arnosera minima (L.) Schweigg. & Koerte 1 folder. Hyoseris minima [TA], Flora Scotica ed. 1, 442 (1777). i. "Hyoseris minima, near Langle